In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, accuracy_score, mean_absolute_error, r2_score
from sklearn.cluster import KMeans
import tensorflow as tf
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Input, Dropout, Bidirectional, Layer
from keras.regularizers import l1_l2
from keras.optimizers import Adam
import warnings
warnings.filterwarnings('ignore')

# Set paths (same as previous)
ROOT = os.getcwd()
DATA_DIR = os.path.join(ROOT, "data")
ENRICHED_DIR = os.path.join(DATA_DIR, "enriched")
MODEL_RESULTS_DIR = os.path.join(DATA_DIR, 'model_results')
os.makedirs(MODEL_RESULTS_DIR, exist_ok=True)

# Commodity names
commodities = ["Gold", "WTI", "Wheat", "NaturalGas", "Copper", "Lithium"]

# Load enriched data
merged_data = {}
for name in commodities:
    fname = f"{name.lower()}_enriched.csv"
    path = os.path.join(ENRICHED_DIR, fname)
    if os.path.exists(path):
        df = pd.read_csv(path)
        df['Date'] = pd.to_datetime(df['Date'])
        merged_data[name] = df
    else:
        print(f"Missing enriched file for {name}")

In [2]:
# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def add_lagged_features(df, feature_cols, max_lag=5):
    """Add lagged features with proper handling"""
    df_copy = df.copy()
    for col in feature_cols:
        if col in df_copy.columns:
            for lag in range(1, max_lag + 1):
                df_copy[f'{col}_lag{lag}'] = df_copy[col].shift(lag)
    return df_copy

def create_sequences(data, feature_cols, target_col, seq_length):
    """Create sequences for LSTM"""
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[feature_cols].iloc[i:i+seq_length].values
        y = data[target_col].iloc[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

def compute_max_drawdown(returns):
    """Calculate maximum drawdown from returns series"""
    cumulative = np.cumprod(1 + returns)
    running_max = np.maximum.accumulate(cumulative)
    drawdown = (cumulative - running_max) / running_max
    return np.min(drawdown)

def backtest_volatility_strategy(vol_pred, vol_actual, returns, transaction_cost=0.001):
    """
    Trading strategy based on volatility predictions

    Strategy Logic:
    - When predicted vol > 75th percentile: Reduce position (expect mean reversion)
    - When predicted vol < 25th percentile: Increase position (expect trending)
    - Use volatility forecast errors to adjust signals
    """
    # Normalize predictions
    vol_pred_norm = (vol_pred - np.mean(vol_pred)) / np.std(vol_pred)

    # Generate signals based on volatility regime
    high_vol_threshold = np.percentile(vol_pred, 75)
    low_vol_threshold = np.percentile(vol_pred, 25)

    signals = np.zeros(len(vol_pred))
    signals[vol_pred > high_vol_threshold] = -0.5  # Reduce exposure in high vol
    signals[vol_pred < low_vol_threshold] = 1.0    # Full exposure in low vol
    signals[(vol_pred >= low_vol_threshold) & (vol_pred <= high_vol_threshold)] = 0.5

    # Align returns (shift by 1 to avoid lookahead bias)
    aligned_returns = returns[1:len(signals)+1]
    signals = signals[:len(aligned_returns)]

    # Calculate strategy returns
    strategy_returns = signals * aligned_returns

    # Apply transaction costs
    position_changes = np.abs(np.diff(np.concatenate([[0], signals])))
    transaction_costs = position_changes * transaction_cost
    strategy_returns = strategy_returns - transaction_costs[:len(strategy_returns)]

    # Metrics
    sharpe = np.mean(strategy_returns) / (np.std(strategy_returns) + 1e-10) * np.sqrt(252)
    cumulative_return = np.prod(1 + strategy_returns) - 1
    max_dd = compute_max_drawdown(strategy_returns)
    win_rate = np.mean(strategy_returns > 0)

    # Buy & Hold comparison
    bh_returns = aligned_returns
    bh_sharpe = np.mean(bh_returns) / (np.std(bh_returns) + 1e-10) * np.sqrt(252)

    return {
        'sharpe': sharpe,
        'cumulative_return': cumulative_return * 100,
        'max_drawdown': max_dd * 100,
        'win_rate': win_rate * 100,
        'bh_sharpe': bh_sharpe,
        'alpha': sharpe - bh_sharpe,
        'avg_return_pct': np.mean(strategy_returns) * 100,
        'volatility': np.std(strategy_returns) * 100
    }

def build_robust_lstm(input_shape, use_bidirectional=False):
    """
    Build LSTM with regularization to prevent overfitting

    Key changes from original:
    - Smaller network (less prone to memorization)
    - L1+L2 regularization
    - Huber loss (robust to outliers)
    """
    model = Sequential([
        Input(shape=input_shape),
        LSTM(32, return_sequences=True if use_bidirectional else False,
             kernel_regularizer=l1_l2(l1=0.0001, l2=0.001)),
        Dropout(0.2),
    ])

    if use_bidirectional:
        model.add(LSTM(16, kernel_regularizer=l1_l2(l1=0.0001, l2=0.001)))
        model.add(Dropout(0.2))

    model.add(Dense(8, activation='relu'))
    model.add(Dense(1))

    # Huber loss is less sensitive to outliers than MSE
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='huber',
        metrics=['mae']
    )

    return model

def prepare_gprd_features(df, gprd_col='GPRD', method='percentile'):
    """
    Smart GPRD feature engineering to avoid noise

    Issue: Raw GPRD may be too noisy for LSTM
    Solution: Use regime indicators instead of raw values
    """
    df_copy = df.copy()

    if method == 'percentile':
        # Convert to percentile ranks (0-100)
        df_copy['GPRD_percentile'] = df_copy[gprd_col].rank(pct=True) * 100

        # Binary high-risk regime
        df_copy['GPRD_high_risk'] = (df_copy['GPRD_percentile'] > 75).astype(int)

        # Changes in GPRD (captures shocks)
        df_copy['GPRD_change'] = df_copy[gprd_col].diff()
        df_copy['GPRD_change_ma5'] = df_copy['GPRD_change'].rolling(5).mean()

    elif method == 'zscore':
        # Z-score normalization
        gprd_mean = df_copy[gprd_col].rolling(60).mean()
        gprd_std = df_copy[gprd_col].rolling(60).std()
        df_copy['GPRD_zscore'] = (df_copy[gprd_col] - gprd_mean) / (gprd_std + 1e-10)
        df_copy['GPRD_zscore'] = df_copy['GPRD_zscore'].clip(-3, 3)  # Remove extreme outliers

    return df_copy


In [3]:
# ============================================================================
# MAIN MODELING PIPELINE
# ============================================================================

def run_enhanced_lstm_with_gprd(commodity_name, df, use_gprd=True):
    """
    Run LSTM with optional GPRD integration and full backtesting
    """
    print(f"\n{'='*80}")
    print(f"Processing: {commodity_name} (GPRD: {'ON' if use_gprd else 'OFF'})")
    print(f"{'='*80}")

    # Prepare data
    df = df.copy()
    df = df.sort_values('Date').reset_index(drop=True)

    # CRITICAL FIX: Detect and rename commodity-specific columns
    close_col = None
    for col in df.columns:
        if 'Close' in col or 'close' in col:
            close_col = col
            break

    if close_col is None:
        print(f"ERROR: No Close column found in {df.columns.tolist()}")
        return None

    # Rename to standard name if needed
    if close_col != 'Close':
        df['Close'] = df[close_col]
        print(f"Renamed {close_col} -> Close")

    # Calculate Return and Vol_5 if missing
    if 'Return' not in df.columns:
        df['Return'] = df['Close'].pct_change()
        print("Calculated Return from Close prices")

    if 'Vol_5' not in df.columns:
        df['Vol_5'] = df['Return'].rolling(5).std()
        print("Calculated Vol_5 from Returns")

    # Identify price column (could be 'Close', 'Price', or 'PRICE')
    price_col = 'Close'  # We just ensured this exists above

    # Add technical indicators if not present
    if 'MA_5' not in df.columns:
        df['MA_5'] = df['Close'].rolling(5).mean()
    if 'MA_20' not in df.columns:
        df['MA_20'] = df['Close'].rolling(20).mean()

    # Feature engineering for GPRD
    if use_gprd and 'GPRD' in df.columns:
        df = prepare_gprd_features(df, method='percentile')
        gprd_features = ['GPRD_percentile', 'GPRD_high_risk', 'GPRD_change_ma5']
    else:
        gprd_features = []

    # Base features (always included)
    base_features = ['Return', 'Vol_5', 'MA_5', 'MA_20']

     # Add MA features only if they were successfully created
    if 'MA_5' in df.columns and not df['MA_5'].isna().all():
        base_features.append('MA_5')
    if 'MA_20' in df.columns and not df['MA_20'].isna().all():
        base_features.append('MA_20')

    # Add sentiment if available
    if 'sentiment' in df.columns:
        df['sentiment'].fillna(0, inplace=True)
        base_features.append('sentiment')

    if 'geo_keyword_hits' in df.columns:
        df['geo_keyword_hits'].fillna(0, inplace=True)
        base_features.append('geo_keyword_hits')

    # Create lagged features
    features_to_lag = base_features + gprd_features
    df = add_lagged_features(df, features_to_lag, max_lag=5)

    # Construct feature list
    feature_cols = []
    for feat in features_to_lag:
        feature_cols.extend([f'{feat}_lag{i}' for i in range(1, 6)])

    # Remove features with too many NaNs
    df_clean = df[['Date', 'Vol_5', 'Return'] + feature_cols].copy()
    df_clean = df_clean.dropna()

    print(f"Features ({len(feature_cols)}): {feature_cols[:3]}... (showing first 3)")
    print(f"Clean data shape: {df_clean.shape}")

    if len(df_clean) < 100:
        print(f"Insufficient data after cleaning. Skipping.")
        return None

    # Train/test split
    split_date = pd.to_datetime('2018-01-01')
    train_df = df_clean[pd.to_datetime(df_clean['Date']) < split_date].copy()
    test_df = df_clean[pd.to_datetime(df_clean['Date']) >= split_date].copy()

    print(f"Train: {len(train_df)} samples | Test: {len(test_df)} samples")

    if len(train_df) < 50 or len(test_df) < 20:
        print("Insufficient train/test data. Skipping.")
        return None

    # Target and features
    target = 'Vol_5'

    # Scaling with RobustScaler (less sensitive to outliers)
    scaler = RobustScaler()
    train_features_scaled = scaler.fit_transform(train_df[feature_cols])
    test_features_scaled = scaler.transform(test_df[feature_cols])

    train_scaled_df = pd.DataFrame(train_features_scaled, columns=feature_cols, index=train_df.index)
    test_scaled_df = pd.DataFrame(test_features_scaled, columns=feature_cols, index=test_df.index)

    train_scaled_df[target] = train_df[target].values
    test_scaled_df[target] = test_df[target].values
    train_scaled_df['Return'] = train_df['Return'].values
    test_scaled_df['Return'] = test_df['Return'].values

    # Create sequences
    seq_length = 20
    X_train_seq, y_train_seq = create_sequences(train_scaled_df, feature_cols, target, seq_length)
    X_test_seq, y_test_seq = create_sequences(test_scaled_df, feature_cols, target, seq_length)

    print(f"Sequences - Train: {X_train_seq.shape} | Test: {X_test_seq.shape}")

    if len(X_train_seq) < 20 or len(X_test_seq) < 5:
        print("Insufficient sequences. Skipping.")
        return None

    # Build and train model
    model = build_robust_lstm(input_shape=(seq_length, len(feature_cols)))

    early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=0)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001, verbose=0)

    history = model.fit(
        X_train_seq, y_train_seq,
        epochs=100,
        batch_size=64,
        validation_split=0.2,
        callbacks=[early_stop, reduce_lr],
        verbose=0
    )

    # Predictions
    y_pred_train = model.predict(X_train_seq, verbose=0).squeeze()
    y_pred_test = model.predict(X_test_seq, verbose=0).squeeze()

    # Evaluation metrics
    train_r2 = r2_score(y_train_seq, y_pred_train)
    test_r2 = r2_score(y_test_seq, y_pred_test)
    test_rmse = np.sqrt(mean_squared_error(y_test_seq, y_pred_test))
    test_mae = mean_absolute_error(y_test_seq, y_pred_test)

    print(f"\nModel Performance:")
    print(f"  Train R²: {train_r2:.4f} | Test R²: {test_r2:.4f}")
    print(f"  Test RMSE: {test_rmse:.4f} | Test MAE: {test_mae:.4f}")

    # Extract returns for backtesting
    test_returns = test_scaled_df['Return'].iloc[seq_length:].values

    # Backtest trading strategy
    backtest_results = backtest_volatility_strategy(
        y_pred_test, y_test_seq, test_returns, transaction_cost=0.001
    )

    print(f"\nTrading Strategy Performance:")
    print(f"  Sharpe Ratio: {backtest_results['sharpe']:.3f}")
    print(f"  Alpha (vs B&H): {backtest_results['alpha']:.3f}")
    print(f"  Cumulative Return: {backtest_results['cumulative_return']:.2f}%")
    print(f"  Max Drawdown: {backtest_results['max_drawdown']:.2f}%")
    print(f"  Win Rate: {backtest_results['win_rate']:.2f}%")

    # Compile results
    results = {
        'commodity': commodity_name,
        'use_gprd': use_gprd,
        'n_features': len(feature_cols),
        'train_r2': train_r2,
        'test_r2': test_r2,
        'test_rmse': test_rmse,
        'test_mae': test_mae,
        **backtest_results
    }
    # Visualization
    plot_results(commodity_name, y_test_seq, y_pred_test, test_returns, history, use_gprd)

    return results

def plot_results(commodity, y_true, y_pred, returns, history, use_gprd):
    """Create comprehensive visualization"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(f'{commodity} - LSTM Results (GPRD: {"ON" if use_gprd else "OFF"})', fontsize=16)

    # Plot 1: Predictions vs Actual
    axes[0, 0].plot(y_true[-200:], label='Actual Vol', alpha=0.7, linewidth=2)
    axes[0, 0].plot(y_pred[-200:], label='Predicted Vol', alpha=0.7, linewidth=2)
    axes[0, 0].set_title('Volatility Forecast (Last 200 days)')
    axes[0, 0].legend()
    axes[0, 0].grid(alpha=0.3)

    # Plot 2: Training history
    axes[0, 1].plot(history.history['loss'], label='Train Loss')
    axes[0, 1].plot(history.history['val_loss'], label='Val Loss')
    axes[0, 1].set_title('Training History')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(alpha=0.3)

    # Plot 3: Residuals
    residuals = y_true - y_pred
    axes[1, 0].scatter(y_pred, residuals, alpha=0.3)
    axes[1, 0].axhline(y=0, color='r', linestyle='--')
    axes[1, 0].set_title('Residual Plot')
    axes[1, 0].set_xlabel('Predicted')
    axes[1, 0].set_ylabel('Residuals')
    axes[1, 0].grid(alpha=0.3)

    # Plot 4: Distribution comparison
    axes[1, 1].hist(y_true, bins=30, alpha=0.5, label='Actual', density=True)
    axes[1, 1].hist(y_pred, bins=30, alpha=0.5, label='Predicted', density=True)
    axes[1, 1].set_title('Distribution Comparison')
    axes[1, 1].legend()
    axes[1, 1].grid(alpha=0.3)

    plt.tight_layout()
    plt.savefig(os.path.join(MODEL_RESULTS_DIR, f'{commodity}_lstm_{"gprd" if use_gprd else "nogprd"}.png'), dpi=150)
    plt.show()


In [4]:
 # ============================================================================
# RUN EXPERIMENTS
# ============================================================================

if __name__ == "__main__":
    # Load data
    print("Loading enriched data...")
    merged_data = {}
    for name in commodities:
        fname = f"{name.lower()}_enriched.csv"
        path = os.path.join(ENRICHED_DIR, fname)
        if os.path.exists(path):
            df = pd.read_csv(path)
            df['Date'] = pd.to_datetime(df['Date'])
            merged_data[name] = df
            print(f"  Loaded {name}: {len(df)} rows, Columns: {df.columns.tolist()[:5]}...")
        else:
            print(f"  Missing: {name}")

    # Run experiments: WITH and WITHOUT GPRD
    all_results = []

    for commodity in merged_data.keys():
        # Without GPRD
        results_no_gprd = run_enhanced_lstm_with_gprd(commodity, merged_data[commodity], use_gprd=False)
        if results_no_gprd:
            all_results.append(results_no_gprd)

        # With GPRD (if available)
        if 'GPRD' in merged_data[commodity].columns:
            results_with_gprd = run_enhanced_lstm_with_gprd(commodity, merged_data[commodity], use_gprd=True)
            if results_with_gprd:
                all_results.append(results_with_gprd)

    # Save results
    results_df = pd.DataFrame(all_results)
    results_df.to_csv(os.path.join(MODEL_RESULTS_DIR, 'lstm_comparison_gprd.csv'), index=False)

    print("\n" + "="*80)
    print("SUMMARY: GPRD Impact Analysis")
    print("="*80)
    print(results_df[['commodity', 'use_gprd', 'test_r2', 'sharpe', 'alpha', 'cumulative_return']])

    # Statistical comparison
    print("\n" + "="*80)
    print("Average Performance by GPRD Usage:")
    print("="*80)
    print(results_df.groupby('use_gprd')[['test_r2', 'sharpe', 'alpha']].mean())

Loading enriched data...
  Loaded Gold: 5342 rows, Columns: ['Date', 'Close_GC=F', 'High_GC=F', 'Low_GC=F', 'Open_GC=F']...
  Loaded WTI: 5351 rows, Columns: ['Date', 'Close_CL=F', 'High_CL=F', 'Low_CL=F', 'Open_CL=F']...
  Loaded Wheat: 5367 rows, Columns: ['Date', 'Close_ZW=F', 'High_ZW=F', 'Low_ZW=F', 'Open_ZW=F']...
  Loaded NaturalGas: 3694 rows, Columns: ['Date', 'Close_UNG', 'High_UNG', 'Low_UNG', 'Open_UNG']...
  Loaded Copper: 5346 rows, Columns: ['Date', 'Close_HG=F', 'High_HG=F', 'Low_HG=F', 'Open_HG=F']...
  Loaded Lithium: 2871 rows, Columns: ['Date', 'Close_LIT', 'High_LIT', 'Low_LIT', 'Open_LIT']...

Processing: Gold (GPRD: OFF)
Renamed Close_GC=F -> Close
Features (40): ['Return_lag1', 'Return_lag2', 'Return_lag3']... (showing first 3)
Clean data shape: (5318, 43)
Train: 4311 samples | Test: 1007 samples


ValueError: Shape of passed values is (4311, 60), indices imply (4311, 40)